In [1]:
import pandas as pd

from leagueGetData import getLeagueData
from leagueDataCleaner import cleanLeagueData

The players' information.

In [2]:
targets = pd.DataFrame(columns=['pseudo', 'tagline', 'region', 'birthDate'], data=[['Katiounette', 'KITTY', 'europe', '01/10/2001'], ['OhSS', '6121', 'europe', '01/01/2000']])
targets

,pseudo,tagline,region,birthDate
0,Katiounette,KITTY,europe,01/10/2001
1,OhSS,6121,europe,01/01/2000


First, I retrieve game data for one or more player(s). The match list and the data for each match is saved for each player in data/matchLists and data/parsedData. (The path can be modified)

Here, I get data for the first time and it's saved on the computer.

In [3]:
targetID = getLeagueData(targets)
targetID

ConnectTimeout: HTTPSConnectionPool(host='europe.api.riotgames.com', port=443): Max retries exceeded with url: /riot/account/v1/accounts/by-riot-id/Katiounette/KITTY (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000202D1D49210>, 'Connection to europe.api.riotgames.com timed out. (connect timeout=None)'))

Here, the data is retrieved from the computer.

In [ ]:
targetID = getLeagueData(targets)
targetID

Got parsed data from os for Katiounette#KITTY


,puuid,pseudo,tagline,region
0,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,Katiounette,KITTY,europe


I join my two DataFrames.

In [ ]:
players = pd.merge(targets, targetID, on=['pseudo', 'tagline', 'region'])
players

,pseudo,tagline,region,birthDate,puuid
0,Katiounette,KITTY,europe,01/10/2001,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...


Then, I clean the data to get a DataFrame with one line per player.

In [ ]:
cleanLeagueData(players)

,pseudo,tagline,region,birthDate,puuid,dayMostFreq,hourMostFreq,monday,tuesday,wednesday,...,primaryVar32,primaryVar13,primaryVar23,primaryVar33,secondaryVar10,secondaryVar20,secondaryVar30,secondaryVar11,secondaryVar21,secondaryVar31
0,Katiounette,KITTY,europe,01/10/2001,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,6.0,2.0,0.168675,0.120482,0.120482,...,0.0,303.409639,0.168675,0.0,105.180723,5.445783,1.566265,72.674699,29.144578,0.0
